<a href="https://colab.research.google.com/github/Aisnya975/problemset_aisyah_033/blob/main/week11_soal19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import math

# Placeholder for conceptual constants/classes/functions
# These would need actual implementations for the code to run meaningfully.
class Node:
    def __init__(self, name, type, children=None, location_pointer=None, next_pointer=None):
        self.name = name
        self.type = type
        self.children = children if children is not None else []
        self.location_pointer = location_pointer
        self.next_pointer = next_pointer

    def __repr__(self):
        return f"Node(name='{self.name}', type='{self.type}', children_count={len(self.children) if self.children else 0})"

# Assuming 'FILE' and 'DIRECTORY' are type identifiers
FILE = "file"
DIRECTORY = "directory"

# Placeholder for disk operations and bitmap management
POINTER_SIZE_BYTES = 4 # Example value for a pointer size

def GET_FREE_BLOCKS(bitmap, total_needed):
    # This function would typically interact with a bitmap to find free blocks.
    # For demonstration, it returns a dummy list of block addresses.
    # 'bitmap' is a conceptual representation here.
    print(f"DEBUG: Getting {total_needed} free blocks from bitmap (conceptual).")
    # In a real system, this would find actual free block indices.
    return list(range(100, 100 + total_needed)) # Dummy addresses

def MARK_USED(bitmap, blocks_list):
    # This function would mark blocks as used in the bitmap.
    print(f"DEBUG: Marking blocks {blocks_list} as used in bitmap (conceptual).")
    pass # No actual bitmap manipulation in this placeholder

def WRITE_TO_DISK(block_address, data):
    # This function would simulate writing data to a physical disk block.
    # 'block_address' would be the address, 'data' would be the content.
    print(f"DEBUG: Writing {data} to disk block {block_address} (conceptual).")
    pass # No actual disk write in this placeholder

def READ_DISK(block_address):
    # This function would simulate reading data from a physical disk block.
    # It's simplified here to return a mock block object for linked list traversal.
    print(f"DEBUG: Reading from disk block {block_address} (conceptual).")
    # In a real system, this would return the actual content of the block.
    # For linked list, it returns an object with a next_pointer.
    class MockBlock:
        def __init__(self, next_ptr_addr):
            self.next_pointer = next_ptr_addr
            # Add other block content here if needed
        def __repr__(self):
            return f"MockBlock(next_pointer={self.next_pointer})"

    # Simplified logic for next_pointer: if address < 120, point to next; else, None
    next_ptr_for_mock = block_address + 1 if block_address < 120 else None
    return MockBlock(next_ptr_for_mock)

def READ_BYTE_AT(block_content, offset):
    # This function would read a specific byte from the given block content.
    print(f"DEBUG: Reading byte at offset {offset} from block {block_content} (conceptual).")
    return b'X' # Return a dummy byte

# --- Corrected Functions --- (Pseudocode translated to Python)

def FindFile(root_directory, full_path):
    # Input: "/" (root node), "/home/student/report.pdf"

    # Split the path, filtering out empty strings that might result from leading/trailing slashes
    parts = [p for p in full_path.split("/") if p]

    current_node = root_directory

    for part in parts:
        found = False

        # Search in the children of the current node
        for child in current_node.children:
            if child.name == part:
                current_node = child
                found = True
                break

        if not found:
            return "Error: Not Found"

    # If the loop completes, current_node is the target file or directory
    if current_node.type == FILE:
        return current_node.location_pointer
    else:
        return "Error: Path points to a directory, not a file"

# Complexity: O(d * n)
# d = directory depth, n = average number of files per folder


def AllocateIndexedFile(file_size_bytes, block_size, bitmap):
    # Calculate the number of data blocks needed
    needed_blocks = math.ceil(file_size_bytes / block_size)

    # Check availability (data blocks + 1 index block)
    total_needed = needed_blocks + 1
    free_blocks_list = GET_FREE_BLOCKS(bitmap, total_needed)

    if len(free_blocks_list) < total_needed:
        return "Error: Not enough space"

    # Take the first block to be the Index Block
    index_block_addr = free_blocks_list[0]
    data_blocks_addr = free_blocks_list[1:] # Python list slicing

    # Update Bitmap (mark as used)
    MARK_USED(bitmap, free_blocks_list)

    # Physically write data pointers into the Index Block
    WRITE_TO_DISK(index_block_addr, data_blocks_addr)

    # Directory update to record the new file is assumed to be handled by the caller
    # based on the returned index_block_addr.
    return index_block_addr

# Edge Cases: Disk full, File size 0, File size exceeds index block pointer limit.


def ReadByteLinked(start_block, byte_offset, block_size):
    # Calculate which logical block the byte is in
    # Deduct pointer size because each block stores a pointer to the next
    effective_data_size = block_size - POINTER_SIZE_BYTES

    # Ensure effective_data_size is positive to avoid issues with division/modulo
    if effective_data_size <= 0:
        return "Error: Block size too small or POINTER_SIZE_BYTES too large for effective data storage"

    target_block_index = math.floor(byte_offset / effective_data_size)
    offset_within_block = byte_offset % effective_data_size

    current_block_obj = start_block # Renamed to avoid confusion if start_block was just an address

    # Traverse the linked chain
    for i in range(target_block_index):
        # Check if the current block object has a 'next_pointer' attribute and it's not None
        if hasattr(current_block_obj, 'next_pointer') and current_block_obj.next_pointer is not None:
            current_block_obj = READ_DISK(current_block_obj.next_pointer)
        else:
            # If we run out of links before reaching the target block, it's an error
            return "Error: Unexpected End of File (next_pointer missing or None)"

    # Now we are in the correct block
    # Read the byte at the calculated offset within this block
    data = READ_BYTE_AT(current_block_obj, offset_within_block)
    return data

# Complexity: O(n) where n is the block position (linear access).